In [1]:
### Install requirements

# Uncomment to install missing pre-requisites
#!pip install elasticdeform
#!pip install tensorflow_addons
#!pip install nibabel
#!pip install matplotlib
#!pip install sklearn

In [4]:
### Imports

# System
import glob
import time
import os
import concurrent.futures
import importlib

# Visualization
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf

# Numerical calculations
import numpy as np
from sklearn.model_selection import train_test_split

# K fold validation
from sklearn.model_selection import KFold

# Own stuff
import config as conf
importlib.reload(conf)

from helper_functions import scan_loader
from model.data_generator import DataGenerator
from model import architecture

from helper_functions.print2file import *
if conf.log_file is not None:
    with open(conf.log_file, "w"):
        pass

In [3]:
# GPU Setup

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = conf.gpu

# Tensorflow 2.XX\n",
allow_multi_gpu = True
tf_version = 2

if tf_version == 2 and allow_multi_gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print2file(f"Num GPUs Available: {len(gpus)}")
    if gpus:
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)

Num GPUs Available: 1


In [6]:
### Loads training/validation data

# Fetches list of image paths for each modality and segmentation ground truth
path_train = conf.aug_export_path + "/*/"

t1_list_train    = sorted(glob.glob(path_train + '*t1.nii.gz'))
t2_list_train    = sorted(glob.glob(path_train + '*t2.nii.gz'))
t1ce_list_train  = sorted(glob.glob(path_train + '*t1ce.nii.gz'))
flair_list_train = sorted(glob.glob(path_train + '*flair.nii.gz'))
seg_list_train   = sorted(glob.glob(path_train + '*seg.nii.gz'))

# Splits data, or loads existing splits.
if conf.make_new_splits and not conf.continue_training:
    ids_list = list(np.arange(len(t1_list_train)))

    # Splits off random 20% as test data
    idxTrain, idxValid =  train_test_split(ids_list, test_size = 0.2, shuffle = True)
    idxTrain = sorted(idxTrain)
    idxValid = sorted(idxValid)

    # Saves indices to numpy files
    np.save(conf.output_path + "/idxValid.npy", idxValid)
    np.save(conf.output_path + "/idxTrain.npy", idxTrain)
else:
    # Loads indices from numpy files
    idxTrain = np.load(conf.output_path + '/idxTrain.npy')
    idxValid = np.load(conf.output_path + '/idxValid.npy')
    
# Compiles the lists from all modalities into one set
sets = {'train': [], 'valid': []}

for i in idxTrain:
    sets['train'].append([t1_list_train[i], t2_list_train[i], t1ce_list_train[i], flair_list_train[i], seg_list_train[i]])
for i in idxValid:
    sets['valid'].append([t1_list_train[i], t2_list_train[i], t1ce_list_train[i], flair_list_train[i], seg_list_train[i]])
    
# Debug info
num_data = len(idxValid) + len(idxTrain)

print2file(f"Num validation data: {len(idxValid)} ({len(idxValid)/num_data*100}%)")
print2file(f"Num training data: {len(idxTrain)} ({len(idxTrain)/num_data*100}%)")

Num validation data: 148 (20.05420054200542%)
Num training data: 590 (79.94579945799458%)


In [10]:
if conf.kfold_validation:
    kf = KFold(n_splits=5, shuffle=True, random_state=12)
    fold_num = 0
    for train_index, valid_index in kf.split(t1_list_train):
        fold_num += 1
        # print("TRAIN:", train_index, "TEST:", test_index)
        np.save(f"{conf.output_path}/idxValid_{fold_num}.npy", valid_index)
        np.save(f"{conf.output_path}/idxTrain_{fold_num}.npy", train_index)
        

In [5]:
### Data Generator Initialization

# Training data generator
train_gen = DataGenerator(sets['train'], 
                          shuffle    = True,
                          input_dim  = conf.augmented_dim,
                          output_dim = conf.model_dim,
                          batch_size = conf.batch_size,
                          n_channels = conf.num_channels,
                          n_classes  = conf.num_classes)

# Validation data generator
valid_gen = DataGenerator(sets['valid'], 
                          shuffle    = True,
                          input_dim  = conf.augmented_dim,
                          output_dim = conf.model_dim,
                          batch_size = conf.batch_size,
                          n_channels = conf.num_channels,
                          n_classes  = conf.num_classes)

## GAN: Vox2Vox

In [6]:
### Model Initialization

# Input shape for images and ground truth:
im_shape = (*conf.model_dim, conf.num_channels) 
gt_shape = (*conf.model_dim, conf.num_classes)

# Loads initial class weights.
class_weights = np.load('resources/class_weights.npy')

# Initializes the model.
gan = architecture.vox2vox(im_shape, gt_shape, class_weights,
                           output_path = conf.output_path, 
                           save_images = conf.export_images) 

# Loads existing model weights.
if conf.continue_training:
    gan.generator.load_weights(conf.output_path + '/Generator.h5')
    gan.discriminator.load_weights(conf.output_path + '/Discriminator.h5')
    gan.combined.load_weights(conf.output_path + '/Vox2Vox.h5') 

In [7]:
### Model training!
start_time = time.time()

trends_train, trends_valid = gan.train(train_gen, valid_gen, conf.num_epochs)

duration = time.time() - start_time
print2file(f"Training duration: {duration} seconds")

Training process:
Training on 148 and validating on 37 batches.

Epoch 1/100
Training Batch: 1/148 - v2v_loss: 0.5304
Training Batch: 2/148 - v2v_loss: 0.5313
Training Batch: 3/148 - v2v_loss: 0.5194
Training Batch: 4/148 - v2v_loss: 0.4454
Training Batch: 5/148 - v2v_loss: 0.7050
Training Batch: 6/148 - v2v_loss: 0.5832
Training Batch: 7/148 - v2v_loss: 0.5299
Training Batch: 8/148 - v2v_loss: 0.6583
Training Batch: 9/148 - v2v_loss: 0.6572
Training Batch: 10/148 - v2v_loss: 0.5207
Training Batch: 11/148 - v2v_loss: 0.5450
Training Batch: 12/148 - v2v_loss: 0.5576
Training Batch: 13/148 - v2v_loss: 0.6086
Training Batch: 14/148 - v2v_loss: 0.7066
Training Batch: 15/148 - v2v_loss: 0.6245
Training Batch: 16/148 - v2v_loss: 0.5342
Training Batch: 17/148 - v2v_loss: 0.5683
Training Batch: 18/148 - v2v_loss: 0.6214
Training Batch: 19/148 - v2v_loss: 0.4957
Training Batch: 20/148 - v2v_loss: 0.6202
Training Batch: 21/148 - v2v_loss: 0.6936
Training Batch: 22/148 - v2v_loss: 0.5973
Training

Training Batch: 4/148 - v2v_loss: 0.7492
Training Batch: 5/148 - v2v_loss: 0.5864
Training Batch: 6/148 - v2v_loss: 0.5046
Training Batch: 7/148 - v2v_loss: 0.5318
Training Batch: 8/148 - v2v_loss: 0.8578
Training Batch: 9/148 - v2v_loss: 0.6035
Training Batch: 10/148 - v2v_loss: 0.6021
Training Batch: 11/148 - v2v_loss: 0.5796
Training Batch: 12/148 - v2v_loss: 0.6267
Training Batch: 13/148 - v2v_loss: 0.6681
Training Batch: 14/148 - v2v_loss: 0.5499
Training Batch: 15/148 - v2v_loss: 0.6479
Training Batch: 16/148 - v2v_loss: 0.6003
Training Batch: 17/148 - v2v_loss: 0.5917
Training Batch: 18/148 - v2v_loss: 0.5932
Training Batch: 19/148 - v2v_loss: 0.7014
Training Batch: 20/148 - v2v_loss: 0.8528
Training Batch: 21/148 - v2v_loss: 0.6463
Training Batch: 22/148 - v2v_loss: 0.5448
Training Batch: 23/148 - v2v_loss: 0.7766
Training Batch: 24/148 - v2v_loss: 0.5414
Training Batch: 25/148 - v2v_loss: 0.5628
Training Batch: 26/148 - v2v_loss: 0.5358
Training Batch: 27/148 - v2v_loss: 0.500

Training Batch: 9/148 - v2v_loss: 0.4585
Training Batch: 10/148 - v2v_loss: 0.5552
Training Batch: 11/148 - v2v_loss: 0.5583
Training Batch: 12/148 - v2v_loss: 0.5853
Training Batch: 13/148 - v2v_loss: 0.6871
Training Batch: 14/148 - v2v_loss: 0.6606
Training Batch: 15/148 - v2v_loss: 0.7398
Training Batch: 16/148 - v2v_loss: 0.5033
Training Batch: 17/148 - v2v_loss: 0.4646
Training Batch: 18/148 - v2v_loss: 0.5970
Training Batch: 19/148 - v2v_loss: 0.6590
Training Batch: 20/148 - v2v_loss: 0.6251
Training Batch: 21/148 - v2v_loss: 0.5979
Training Batch: 22/148 - v2v_loss: 0.6135
Training Batch: 23/148 - v2v_loss: 0.5889
Training Batch: 24/148 - v2v_loss: 0.5867
Training Batch: 25/148 - v2v_loss: 0.5953
Training Batch: 26/148 - v2v_loss: 0.5171
Training Batch: 27/148 - v2v_loss: 0.7991
Training Batch: 28/148 - v2v_loss: 0.6152
Training Batch: 29/148 - v2v_loss: 0.4733
Training Batch: 30/148 - v2v_loss: 0.5867
Training Batch: 31/148 - v2v_loss: 0.6365
Training Batch: 32/148 - v2v_loss: 

Training Batch: 15/148 - v2v_loss: 0.6408
Training Batch: 16/148 - v2v_loss: 0.6093
Training Batch: 17/148 - v2v_loss: 0.6268
Training Batch: 18/148 - v2v_loss: 0.6045
Training Batch: 19/148 - v2v_loss: 0.6121
Training Batch: 20/148 - v2v_loss: 0.6362
Training Batch: 21/148 - v2v_loss: 0.5318
Training Batch: 22/148 - v2v_loss: 0.5714
Training Batch: 23/148 - v2v_loss: 0.4726
Training Batch: 24/148 - v2v_loss: 0.5653
Training Batch: 25/148 - v2v_loss: 0.7023
Training Batch: 26/148 - v2v_loss: 0.5578
Training Batch: 27/148 - v2v_loss: 0.5864
Training Batch: 28/148 - v2v_loss: 0.6486
Training Batch: 29/148 - v2v_loss: 0.5438
Training Batch: 30/148 - v2v_loss: 0.5631
Training Batch: 31/148 - v2v_loss: 0.7058
Training Batch: 32/148 - v2v_loss: 0.5843
Training Batch: 33/148 - v2v_loss: 0.6145
Training Batch: 34/148 - v2v_loss: 0.6035
Training Batch: 35/148 - v2v_loss: 0.6311
Training Batch: 36/148 - v2v_loss: 0.7080
Training Batch: 37/148 - v2v_loss: 0.6598
Training Batch: 38/148 - v2v_loss:

Training Batch: 21/148 - v2v_loss: 0.5024
Training Batch: 22/148 - v2v_loss: 0.6188
Training Batch: 23/148 - v2v_loss: 0.6775
Training Batch: 24/148 - v2v_loss: 0.5335
Training Batch: 25/148 - v2v_loss: 0.5932
Training Batch: 26/148 - v2v_loss: 0.5782
Training Batch: 27/148 - v2v_loss: 0.5539
Training Batch: 28/148 - v2v_loss: 0.5947
Training Batch: 29/148 - v2v_loss: 0.7554
Training Batch: 30/148 - v2v_loss: 0.5589
Training Batch: 31/148 - v2v_loss: 0.7156
Training Batch: 32/148 - v2v_loss: 0.7491
Training Batch: 33/148 - v2v_loss: 0.6394
Training Batch: 34/148 - v2v_loss: 0.6600
Training Batch: 35/148 - v2v_loss: 0.6005
Training Batch: 36/148 - v2v_loss: 0.5526
Training Batch: 37/148 - v2v_loss: 0.5498
Training Batch: 38/148 - v2v_loss: 0.7902
Training Batch: 39/148 - v2v_loss: 0.6606
Training Batch: 40/148 - v2v_loss: 0.5677
Training Batch: 41/148 - v2v_loss: 0.6301
Training Batch: 42/148 - v2v_loss: 0.6820
Training Batch: 43/148 - v2v_loss: 0.6042
Training Batch: 44/148 - v2v_loss:

...

Training Batch: 61/148 - v2v_loss: 0.5070
Training Batch: 62/148 - v2v_loss: 0.5145
Training Batch: 63/148 - v2v_loss: 0.5010
Training Batch: 64/148 - v2v_loss: 0.5438
Training Batch: 65/148 - v2v_loss: 0.4852
Training Batch: 66/148 - v2v_loss: 0.7003
Training Batch: 67/148 - v2v_loss: 0.6330
Training Batch: 68/148 - v2v_loss: 0.4594
Training Batch: 69/148 - v2v_loss: 0.6258
Training Batch: 70/148 - v2v_loss: 0.4988
Training Batch: 71/148 - v2v_loss: 0.5149
Training Batch: 72/148 - v2v_loss: 0.4405
Training Batch: 73/148 - v2v_loss: 0.6726
Training Batch: 74/148 - v2v_loss: 0.5000
Training Batch: 75/148 - v2v_loss: 0.5276
Training Batch: 76/148 - v2v_loss: 0.5481
Training Batch: 77/148 - v2v_loss: 0.5720
Training Batch: 78/148 - v2v_loss: 0.5602
Training Batch: 79/148 - v2v_loss: 0.4784
Training Batch: 80/148 - v2v_loss: 0.5631
Training Batch: 81/148 - v2v_loss: 0.6471
Training Batch: 82/148 - v2v_loss: 0.4967
Training Batch: 83/148 - v2v_loss: 0.4262
Training Batch: 84/148 - v2v_loss:

Training Batch: 67/148 - v2v_loss: 0.4687
Training Batch: 68/148 - v2v_loss: 0.6035
Training Batch: 69/148 - v2v_loss: 0.6216
Training Batch: 70/148 - v2v_loss: 0.4501
Training Batch: 71/148 - v2v_loss: 0.5618
Training Batch: 72/148 - v2v_loss: 0.5583
Training Batch: 73/148 - v2v_loss: 0.5776
Training Batch: 74/148 - v2v_loss: 0.5878
Training Batch: 75/148 - v2v_loss: 0.4655
Training Batch: 76/148 - v2v_loss: 0.4826
Training Batch: 77/148 - v2v_loss: 0.5159
Training Batch: 78/148 - v2v_loss: 0.5719
Training Batch: 79/148 - v2v_loss: 0.5222
Training Batch: 80/148 - v2v_loss: 0.6400
Training Batch: 81/148 - v2v_loss: 0.6855
Training Batch: 82/148 - v2v_loss: 0.5229
Training Batch: 83/148 - v2v_loss: 0.5987
Training Batch: 84/148 - v2v_loss: 0.5551
Training Batch: 85/148 - v2v_loss: 0.5664
Training Batch: 86/148 - v2v_loss: 0.5489
Training Batch: 87/148 - v2v_loss: 0.4592
Training Batch: 88/148 - v2v_loss: 0.5442
Training Batch: 89/148 - v2v_loss: 0.5162
Training Batch: 90/148 - v2v_loss:

Training Batch: 73/148 - v2v_loss: 0.5346
Training Batch: 74/148 - v2v_loss: 0.5195
Training Batch: 75/148 - v2v_loss: 0.5229
Training Batch: 76/148 - v2v_loss: 0.6584
Training Batch: 77/148 - v2v_loss: 0.5847
Training Batch: 78/148 - v2v_loss: 0.4773
Training Batch: 79/148 - v2v_loss: 0.6982
Training Batch: 80/148 - v2v_loss: 0.5684
Training Batch: 81/148 - v2v_loss: 0.5381
Training Batch: 82/148 - v2v_loss: 0.6633
Training Batch: 83/148 - v2v_loss: 0.5191
Training Batch: 84/148 - v2v_loss: 0.6226
Training Batch: 85/148 - v2v_loss: 0.6053
Training Batch: 86/148 - v2v_loss: 0.5515
Training Batch: 87/148 - v2v_loss: 0.5488
Training Batch: 88/148 - v2v_loss: 0.5684
Training Batch: 89/148 - v2v_loss: 0.5128
Training Batch: 90/148 - v2v_loss: 0.5754
Training Batch: 91/148 - v2v_loss: 0.5865
Training Batch: 92/148 - v2v_loss: 0.5689
Training Batch: 93/148 - v2v_loss: 0.5608
Training Batch: 94/148 - v2v_loss: 0.5149
Training Batch: 95/148 - v2v_loss: 0.5357
Training Batch: 96/148 - v2v_loss:

Training Batch: 79/148 - v2v_loss: 0.5546
Training Batch: 80/148 - v2v_loss: 0.5148
Training Batch: 81/148 - v2v_loss: 0.5331
Training Batch: 82/148 - v2v_loss: 0.5397
Training Batch: 83/148 - v2v_loss: 0.5963
Training Batch: 84/148 - v2v_loss: 0.4924
Training Batch: 85/148 - v2v_loss: 0.6791
Training Batch: 86/148 - v2v_loss: 0.5900
Training Batch: 87/148 - v2v_loss: 0.5528
Training Batch: 88/148 - v2v_loss: 0.4152
Training Batch: 89/148 - v2v_loss: 0.4855
Training Batch: 90/148 - v2v_loss: 0.5332
Training Batch: 91/148 - v2v_loss: 0.5552
Training Batch: 92/148 - v2v_loss: 0.6185
Training Batch: 93/148 - v2v_loss: 0.5199
Training Batch: 94/148 - v2v_loss: 0.5127
Training Batch: 95/148 - v2v_loss: 0.5253
Training Batch: 96/148 - v2v_loss: 0.5539
Training Batch: 97/148 - v2v_loss: 0.5108
Training Batch: 98/148 - v2v_loss: 0.5539
Training Batch: 99/148 - v2v_loss: 0.5974
Training Batch: 100/148 - v2v_loss: 0.5525
Training Batch: 101/148 - v2v_loss: 0.5125
Training Batch: 102/148 - v2v_lo

KeyboardInterrupt: 

In [ ]:
### Simple training evaluation

# Plots loss over epochs.
train_losses = np.load(conf.output_path + '/history_train.npy', allow_pickle=True).tolist()
valid_losses = np.load(conf.output_path + '/history_valid.npy', allow_pickle=True).tolist()

# train_losses = train_losses['loss']
# valid_losses = valid_losses['loss']

# Gets 10 smallest losses from the history.
num_min = min(10, len(train_losses)-1)
min_loss_indices = np.argpartition(valid_losses, num_min)[:num_min]

print2file(f'Minimum validation loss: Epoch: {np.argmin(valid_losses)} - Loss: {np.min(valid_losses)}')
print2file([f"Epoch: {e} - Loss: {round(valid_losses[e],2)}" for e in min_loss_indices])

# Shows plot for training and validation loss.
plt.plot(train_losses, label="Training Loss")
plt.plot(valid_losses, label="Validation Loss")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

In [ ]:
### Saves the model settings for the segmentation module.

import json

#        self.model_dim = config["model_dim"]
#        self.model_path = config["model_path"]
#        self.class_weights = config["class_weights"]
#        self.num_classes = config["num_classes"]
#        self.num_modalities = config["num_modalities"]

seg_config = {
    "model_dim": conf.model_dim,
    "model_path": "/mnt/Data/Vox2vox_output/Generator.h5",
    "class_weights": class_weights.tolist(),
    "num_classes": conf.num_classes,
    "num_modalities": conf.num_channels
}

with open(conf.output_path + "/seg.json", "w") as seg_config_file:
    json.dump(seg_config, seg_config_file, indent=4)
    
print("Done")